In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/815.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/491.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/718.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/709.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/379.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/780.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/248.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/480.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/236.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/630.jpg_0.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/771.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/675.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/340.png
/kaggle/input/drowsy-detection-dataset/Drowsy

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

# **ÿ™ÿπÿ±ŸäŸÅ ÿßŸÑÿ´Ÿàÿßÿ®ÿ™**
SEQUENCE_LENGTH = 15  # ÿπÿØÿØ ÿßŸÑÿ•ÿ∑ÿßÿ±ÿßÿ™ ŸÑŸÉŸÑ ÿ™ÿ≥ŸÑÿ≥ŸÑ
IMG_SIZE = (224, 224)  # ÿ≠ÿ¨ŸÖ ÿßŸÑÿµŸàÿ±ÿ© ŸÑŸÉŸÑ ÿ•ÿ∑ÿßÿ±
BATCH_SIZE = 8  # ÿπÿØÿØ ÿßŸÑÿ™ÿ≥ŸÑÿ≥ŸÑÿßÿ™ ŸÅŸä ŸÉŸÑ batch (Ÿäÿ¨ÿ® ÿ£ŸÜ ŸäŸÉŸàŸÜ ŸÖÿ∂ÿßÿπŸÅŸãÿß ŸÑŸÄ SEQUENCE_LENGTH)
IMAGES_PER_BATCH = SEQUENCE_LENGTH * BATCH_SIZE  # ÿ•ÿ¨ŸÖÿßŸÑŸä ÿπÿØÿØ ÿßŸÑÿµŸàÿ± ŸÅŸä ŸÉŸÑ batch

# **ÿ™ÿ≠ŸÖŸäŸÑ ÿßŸÑÿ®ŸäÿßŸÜÿßÿ™**
data_dir = "/kaggle/input/drowsy-detection-dataset/Drowsy_datset/"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=IMAGES_PER_BATCH,  # ÿ™ÿ≠ŸÖŸäŸÑ ÿπÿØÿØ ŸÖŸÜÿßÿ≥ÿ® ŸÖŸÜ ÿßŸÑÿµŸàÿ±
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=IMAGES_PER_BATCH,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

# **ÿ™ÿ≠ŸàŸäŸÑ ÿßŸÑÿµŸàÿ± ÿ•ŸÑŸâ ÿ™ÿ≥ŸÑÿ≥ŸÑÿßÿ™ (Sequences)**
def create_sequence(generator):
    while True:
        batch = next(generator)
        images, labels = batch

        num_samples = images.shape[0]  # ÿπÿØÿØ ÿßŸÑÿµŸàÿ± ÿßŸÑŸÖÿ≠ŸÖŸÑÿ©

        # **ÿ™ÿ£ŸÉÿØ ŸÖŸÜ ÿ£ŸÜ ÿπÿØÿØ ÿßŸÑÿµŸàÿ± ŸäŸÖŸÉŸÜ ÿ™ŸÇÿ≥ŸäŸÖŸá ÿ•ŸÑŸâ ÿ™ÿ≥ŸÑÿ≥ŸÑÿßÿ™**
        if num_samples % SEQUENCE_LENGTH != 0:
            continue  # ÿ™ÿ¨ÿßŸàÿ≤ Ÿáÿ∞Ÿá ÿßŸÑÿØŸàÿ±ÿ© ŸÖŸÜ ÿßŸÑŸÑŸàÿ® ÿ•ÿ∞ÿß ŸÑŸÖ ŸäŸÉŸÜ ÿßŸÑÿπÿØÿØ ŸÖÿ™ŸàÿßŸÅŸÇŸãÿß

        # ÿ™ŸÇÿ≥ŸäŸÖ ÿßŸÑÿµŸàÿ± ÿ•ŸÑŸâ ŸÖÿ¨ŸÖŸàÿπÿßÿ™ ŸÖŸÜ SEQUENCE_LENGTH
        X_seq = np.reshape(images, (-1, SEQUENCE_LENGTH, IMG_SIZE[0], IMG_SIZE[1], 3))
        y_seq = np.reshape(labels, (-1, SEQUENCE_LENGTH))[:, 0]  # ÿ£ÿÆÿ∞ ÿßŸÑÿ™ÿµŸÜŸäŸÅ ÿßŸÑÿ£ŸàŸÑ ŸÅŸÇÿ∑
        
        yield X_seq, y_seq

train_generator = create_sequence(train_gen)
val_generator = create_sequence(val_gen)

Found 4688 images belonging to 2 classes.
Found 1171 images belonging to 2 classes.


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/drowsy-detection-dataset/Drowsy_datset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    '/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

print(f"ÿπÿØÿØ ÿπŸäŸÜÿßÿ™ ÿßŸÑÿ™ÿØÿ±Ÿäÿ®: {train_generator.samples}")
print(f"ÿπÿØÿØ ÿπŸäŸÜÿßÿ™ ÿßŸÑÿ™ÿ≠ŸÇŸÇ: {val_generator.samples}")


Found 5859 images belonging to 2 classes.
Found 1483 images belonging to 2 classes.
ÿπÿØÿØ ÿπŸäŸÜÿßÿ™ ÿßŸÑÿ™ÿØÿ±Ÿäÿ®: 5859
ÿπÿØÿØ ÿπŸäŸÜÿßÿ™ ÿßŸÑÿ™ÿ≠ŸÇŸÇ: 1483


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# **ÿ•ÿπÿØÿßÿØ ÿßŸÑÿ®ŸäÿßŸÜÿßÿ™**
IMG_SIZE = (96, 96)
SEQUENCE_LENGTH = 8
BATCH_SIZE = 4

# ÿ™ÿ≠ŸÖŸäŸÑ ÿßŸÑÿ®ŸäÿßŸÜÿßÿ™ ÿ®ÿßÿ≥ÿ™ÿÆÿØÿßŸÖ ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    "/kaggle/input/drowsy-detection-dataset/Drowsy_datset/train",
    target_size=IMG_SIZE,
    batch_size=1,  # ÿ≥Ÿäÿ™ŸÖ ÿ™ÿ¨ŸÖŸäÿπ ÿßŸÑÿ•ÿ∑ÿßÿ±ÿßÿ™ ŸÑÿßÿ≠ŸÇŸãÿß
    class_mode="binary",
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    "/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test",
    target_size=IMG_SIZE,
    batch_size=1,  # ÿ≥Ÿäÿ™ŸÖ ÿ™ÿ¨ŸÖŸäÿπ ÿßŸÑÿ•ÿ∑ÿßÿ±ÿßÿ™ ŸÑÿßÿ≠ŸÇŸãÿß
    class_mode="binary",
    shuffle=False
)

# ** ÿ•ŸÜÿ¥ÿßÿ° ŸÖŸàŸÑŸëÿØ ŸÖÿÆÿµÿµ ŸÑÿ•ŸÜÿ¥ÿßÿ° ÿßŸÑÿ™ÿ≥ŸÑÿ≥ŸÑÿßÿ™**
def custom_data_generator(image_generator, sequence_length):
    """ŸÖŸàŸÑÿØ ÿ®ŸäÿßŸÜÿßÿ™ ŸäŸÇŸàŸÖ ÿ®ÿ™ÿ¨ŸÖŸäÿπ ÿßŸÑÿ•ÿ∑ÿßÿ±ÿßÿ™ ŸÅŸä ÿ™ÿ≥ŸÑÿ≥ŸÑÿßÿ™"""
    frames, labels = [], []
    
    while True:
        for img, label in image_generator:
            frames.append(img[0])  # ÿßÿ≥ÿ™ÿÆÿ±ÿßÿ¨ ÿßŸÑÿµŸàÿ±ÿ© ŸÖŸÜ ÿßŸÑŸÄ batch
            labels.append(label[0])
            
            if len(frames) == sequence_length:
                yield np.array(frames).reshape(1, sequence_length, IMG_SIZE[0], IMG_SIZE[1], 3), np.array(labels[-1]).reshape(1, 1)
                frames, labels = [], []  # ÿ•ÿπÿßÿØÿ© ÿ™ÿπŸäŸäŸÜ ÿßŸÑŸÇŸàÿßÿ¶ŸÖ

train_gen = custom_data_generator(train_generator, SEQUENCE_LENGTH)
val_gen = custom_data_generator(val_generator, SEQUENCE_LENGTH)

# ** ÿ®ŸÜÿßÿ° ÿßŸÑŸÜŸÖŸàÿ∞ÿ¨**
base_model = MobileNetV2(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), include_top=False, weights='imagenet')
base_model.trainable = False  # ÿ™ÿ¨ŸÖŸäÿØ ÿ£Ÿàÿ≤ÿßŸÜ MobileNetV2

model = Sequential([
    TimeDistributed(base_model, input_shape=(SEQUENCE_LENGTH, IMG_SIZE[0], IMG_SIZE[1], 3)),
    TimeDistributed(GlobalAveragePooling2D()),  # ÿ™ŸÇŸÑŸäŸÑ ÿ≠ÿ¨ŸÖ ÿßŸÑŸÖÿπÿßŸÖŸÑÿßÿ™
    LSTM(32),  # ÿ™ ÿßŸÑ LSTM
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# ** ÿ•ÿπÿØÿßÿØÿßÿ™ ÿßŸÑÿ™ÿØÿ±Ÿäÿ®**
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ** ÿ∑ÿ®ÿßÿπÿ© ŸÖŸÑÿÆÿµ ÿßŸÑŸÜŸÖŸàÿ∞ÿ¨**
model.summary()

# ** ÿ∂ÿ®ÿ∑ ÿßŸÑŸÄ Callbacks**
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5)

# ** ÿ™ÿØÿ±Ÿäÿ® ÿßŸÑŸÜŸÖŸàÿ∞ÿ¨**
model.fit(train_gen,
          validation_data=val_gen,
          epochs=15,
          steps_per_epoch=len(train_generator) // SEQUENCE_LENGTH,
          validation_steps=len(val_generator) // SEQUENCE_LENGTH,
          callbacks=[early_stopping, lr_scheduler])


Found 5859 images belonging to 2 classes.
Found 1483 images belonging to 2 classes.
9406464/9406464 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

‚îè‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚î≥‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚î≥‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îì
‚îÉ Layer (type)                         ‚îÉ Output Shape                ‚îÉ         Param # ‚îÉ
‚î°‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚ïá‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚ïá‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚î©
‚îÇ time_distributed (TimeDistributed)   ‚îÇ (None, 8, 3, 3, 1280)       ‚îÇ       2,257,984 ‚îÇ
‚îú‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îº‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îº‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚î§
‚îÇ time_distributed_1 (TimeDistributed) ‚îÇ (None, 8, 1280)             ‚îÇ               0 ‚îÇ
‚îú‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îº‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îº‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚î§
‚îÇ lstm (LSTM)                          ‚îÇ (None, 32)                  ‚îÇ         168,064 ‚îÇ
‚îú‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îº‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îº‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚î§
‚îÇ dropout (Dropout)                    ‚îÇ (None, 32)                  ‚îÇ               0 ‚îÇ
‚îú‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îº‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îº‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚î§
‚îÇ dense (Dense)                        ‚îÇ (None, 32)                  ‚îÇ           1,056 ‚îÇ
‚îú‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îº‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îº‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚î§
‚îÇ dropout_1 (Dropout)                  ‚îÇ (None, 32)                  ‚îÇ               0 ‚îÇ
‚îú‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îº‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îº‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚î§
‚îÇ dense_1 (Dense)                      ‚îÇ (None, 1)                   ‚îÇ              33 ‚îÇ
‚îî‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚î¥‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚î¥‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îò

 Total params: 2,427,137 (9.26 MB)

 Trainable params: 169,153 (660.75 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/15
732/732 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 75s 59ms/step - accuracy: 0.6526 - loss: 0.6073 - val_accuracy: 0.8973 - val_loss: 0.2511 - learning_rate: 5.0000e-04
Epoch 2/15
732/732 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 31s 43ms/step - accuracy: 0.8494 - loss: 0.3478 - val_accuracy: 0.9514 - val_loss: 0.1766 - learning_rate: 5.0000e-04
Epoch 3/15
686/732 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 1s 35ms/step - accuracy: 0.8918 - loss: 0.2587

In [ ]:
model.save("model.h5")


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# ÿ™ÿ≠ŸÖŸäŸÑ ÿßŸÑŸÜŸÖŸàÿ∞ÿ¨ ÿßŸÑŸÖÿØÿ±ÿ®
model = tf.keras.models.load_model("model.h5")

# ÿ™ÿµŸÜŸäŸÅÿßÿ™ ÿßŸÑŸÅÿ¶ÿßÿ™
class_names = ["ŸÜÿßÿ¶ŸÖ", "ŸÖÿ≥ÿ™ŸäŸÇÿ∏"]

# ÿØÿßŸÑÿ© ŸÑÿ™ŸàŸÇÿπ ÿßŸÑŸÅÿ¶ÿ© ŸÖŸÜ ÿµŸàÿ±ÿ© Ÿàÿßÿ≠ÿØÿ© Ÿàÿπÿ±ÿ∂Ÿáÿß
def predict_and_show_image(image_path):
    # ÿ™ÿ≠ŸÖŸäŸÑ ÿßŸÑÿµŸàÿ±ÿ© Ÿàÿ™ÿ∫ŸäŸäÿ± ÿßŸÑÿ≠ÿ¨ŸÖ
    img = image.load_img(image_path, target_size=(96, 96))
    img_array = image.img_to_array(img) / 255.0  # ÿ™ÿ≠ŸàŸäŸÑ ŸÑŸÖÿµŸÅŸàŸÅÿ© Ÿàÿ™ŸÇÿ≥ŸäŸÖ ÿπŸÑŸâ 255

    # ÿ™ŸÉÿ±ÿßÿ± ÿßŸÑÿµŸàÿ±ÿ© 8 ŸÖÿ±ÿßÿ™ ŸÑÿ¨ÿπŸÑ ÿßŸÑÿ¥ŸÉŸÑ ŸÖÿ™ŸàÿßŸÅŸÇŸãÿß ŸÖÿπ ÿßŸÑŸÜŸÖŸàÿ∞ÿ¨
    img_sequence = np.array([img_array] * 8)  # ÿßŸÑÿ¢ŸÜ ÿ¥ŸÉŸÑŸá (8, 96, 96, 3)
    img_sequence = np.expand_dims(img_sequence, axis=0)  # Ÿäÿµÿ®ÿ≠ (1, 8, 96, 96, 3)

    # ÿ•ÿ¨ÿ±ÿßÿ° ÿßŸÑÿ™ŸàŸÇÿπ
    prediction = model.predict(img_sequence)[0][0]
    label = class_names[int(prediction > 0.5)]
    confidence = prediction if prediction > 0.5 else 1 - prediction

    # ŸÇÿ±ÿßÿ°ÿ© ÿßŸÑÿµŸàÿ±ÿ© ŸÑÿπÿ±ÿ∂Ÿáÿß ÿ®ÿßÿ≥ÿ™ÿÆÿØÿßŸÖ OpenCV
    img_cv = cv2.imread(image_path)
    img_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)  # ÿ™ÿ≠ŸàŸäŸÑ ÿßŸÑÿ£ŸÑŸàÿßŸÜ ŸÖŸÜ BGR ÿ•ŸÑŸâ RGB

    # ÿπÿ±ÿ∂ ÿßŸÑÿµŸàÿ±ÿ© ÿ®ÿßÿ≥ÿ™ÿÆÿØÿßŸÖ matplotlib
    plt.figure(figsize=(5, 5))
    plt.imshow(img_cv)
    plt.axis("off")  # ÿ•ÿÆŸÅÿßÿ° ÿßŸÑŸÖÿ≠ÿßŸàÿ±
    plt.show()

    # **ÿ∑ÿ®ÿßÿπÿ© ÿßŸÑŸÜÿ™Ÿäÿ¨ÿ© ŸÅŸä ÿßŸÑŸÉŸàŸÜÿ≥ŸàŸÑ ÿ®ÿØŸÑŸãÿß ŸÖŸÜ Ÿàÿ∂ÿπŸáÿß ÿπŸÑŸâ ÿßŸÑÿµŸàÿ±ÿ©**
    print(f"üîπ ÿßŸÑÿ™ŸàŸÇÿπ: {label} (ÿ´ŸÇÿ©: {confidence:.2%})")

# **ÿßÿ≥ÿ™ÿÆÿØÿßŸÖ ÿßŸÑŸÉŸàÿØ ŸÑÿßÿÆÿ™ÿ®ÿßÿ± ÿµŸàÿ±ÿ©**
image_path = "/kaggle/input/drowsy-detection-dataset/Drowsy_datset/train/NATURAL/860.png"
predict_and_show_image(image_path)


In [ ]:
image_path = "/kaggle/input/drowsy-detection-dataset/Drowsy_datset/train/DROWSY/1502.png"
predict_and_show_image(image_path)